In [14]:
from convokit import Corpus, politenessStrategies, Parser

In [15]:
iq2 = Corpus(filename="./iq2_corpus")
anno = Parser()
iq2 = anno.fit_transform(iq2)

In [16]:
ps = politenessStrategies.PolitenessStrategies()
ps.transform(iq2)

In [57]:
print(iq2.get_conversation("0"))
def determine_winner(conversation):
    results = conversation.meta["results"]
    fordelta = results["post"]["for"] - results["pre"]["for"]
    againstdelta = results["post"]["against"] - results["pre"]["against"]
    if(fordelta > againstdelta):
        return "for"
    elif(againstdelta > fordelta):
        return "against"
    else:
        return "tie"

def determine_popular(conversation):
    results = conversation.meta["results"]
    forpop = results["pre"]["for"]
    againstpop = results["pre"]["against"]
    if(forpop > againstpop + 20):
        return "for"
    elif(againstpop > forpop + 20):
        return "against"
    else:
        return "no significant advantage"
    

Conversation({'_owner': <convokit.model.corpus.Corpus object at 0x7f76546e39d0>, '_id': '0', '_utterance_ids': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142

In [58]:
pos = set(['feature_politeness_==Please==',
           'feature_politeness_==Indirect_(btw)==',
           'feature_politeness_==Apologizing==',
           'feature_politeness_==Deference==',
           'feature_politeness_==Gratitude==',
           'feature_politeness_==Hedges==',
           'feature_politeness_==1st_person_pl.==',
           'feature_politeness_==1st_person==',
           'feature_politeness_==1st_person_start==', 
           'feature_politeness_==2nd_person==', 
           'feature_politeness_==Indirect_(greeting)==',
           'feature_politeness_==HASHEDGE==', 
           'feature_politeness_==HASPOSITIVE=='
          ])

neg = set(['feature_politeness_==Please_start==',
           'feature_politeness_==Factuality==',
           'feature_politeness_==Direct_question==',
           'feature_politeness_==Direct_start==',
           'feature_politeness_==2nd_person_start==',
           'feature_politeness_==HASNEGATIVE=='
          ])

unsure = set(['feature_politeness_==SUBJUNCTIVE==',
              'feature_politeness_==INDICATIVE=='
             ])

In [59]:
num_winner_utt, num_loser_utt = 0, 0
num_winner_pos, num_loser_pos = 0, 0
num_winner_neg, num_loser_neg = 0, 0

num_pop_utt, num_unpop_utt = 0, 0
num_pop_pos, num_unpop_pos = 0, 0
num_pop_neg, num_unpop_neg = 0, 0

for conv_root in iq2.get_conversation_ids():
    conv = iq2.get_conversation(conv_root)
    winner = determine_winner(conv)
    if winner == "tie":
        print("debate tied; information not useful here")
        pass
    loser = "against" if winner == "for" else "for"
    
    pop = determine_popular(conv)
    if pop == "no significant advantage":
        print("initial popularity not very biased; information not useful here")
        pass
    unpop = "against" if pop == "for" else "for"
    
    for utt_id in conv.get_utterance_ids():
        utt = iq2.get_utterance(utt_id)
        pos_used, neg_used = 0, 0
        for strat in utt.meta["politeness_strategies"].keys():
            if strat in pos:
                pos_used += utt.meta["politeness_strategies"][strat]
            elif strat in neg:
                neg_used += utt.meta["politeness_strategies"][strat]
        if utt.meta["speakertype"] == winner:
            num_winner_utt += 1
            num_winner_pos += pos_used
            num_winner_neg += neg_used
        elif utt.meta["speakertype"] == loser:
            num_loser_utt += 1
            num_loser_pos += pos_used
            num_loser_neg += neg_used
        if utt.meta["speakertype"] == pop:
            num_pop_utt += 1
            num_pop_pos += pos_used
            num_pop_neg += neg_used
        elif utt.meta["speakertype"] == unpop:
            num_unpop_utt += 1
            num_unpop_pos += pos_used
            num_unpop_neg += neg_used

debate tied; information not useful here
initial popularity not very biased; information not useful here
debate tied; information not useful here
initial popularity not very biased; information not useful here
initial popularity not very biased; information not useful here
initial popularity not very biased; information not useful here
initial popularity not very biased; information not useful here
initial popularity not very biased; information not useful here
debate tied; information not useful here
initial popularity not very biased; information not useful here
initial popularity not very biased; information not useful here
initial popularity not very biased; information not useful here
initial popularity not very biased; information not useful here
initial popularity not very biased; information not useful here
initial popularity not very biased; information not useful here
initial popularity not very biased; information not useful here
initial popularity not very biased; informati

In [60]:
print("Average number of polite strategies used per utterance by winner: {:0.2f}".format(num_winner_pos / num_winner_utt) )
print("Average number of impolite strategies used per utterance by winner: {:0.2f}".format(num_winner_neg / num_winner_utt) )
print("Average number of polite strategies used per utterance by loser: {:0.2f}".format(num_loser_pos / num_loser_utt) )
print("Average number of impolite strategies used per utterance by loser: {:0.2f}".format(num_loser_neg / num_loser_utt) )
print("\n\n")
print("Total number of winner utterances: " + str(num_winner_utt))
print("Total number of loser utterances: " + str(num_loser_utt))
print("\n\n")
print("Total number of politeness strategies used in winner utterances: " + str(num_winner_pos))
print("Total number of impoliteness strategies used in winner utterances: " + str(num_winner_neg))
print("Total number of politeness strategies used in loser utterances: " + str(num_loser_pos))
print("Total number of impoliteness strategies used in loser utterances: " + str(num_loser_neg))
print("-------------------------------------------------------------------------")
print("Average number of polite strategies used per utterance by popular side: {:0.2f}".format(num_pop_pos / num_pop_utt) )
print("Average number of impolite strategies used per utterance by popular side: {:0.2f}".format(num_pop_neg / num_pop_utt) )
print("Average number of polite strategies used per utterance by unpopular side: {:0.2f}".format(num_unpop_pos / num_unpop_utt) )
print("Average number of impolite strategies used per utterance by unpopular side: {:0.2f}".format(num_unpop_neg / num_unpop_utt) )
print("\n\n")
print("Total number of popular side utterances: " + str(num_pop_utt))
print("Total number of unpopular side utterances: " + str(num_unpop_utt))
print("\n\n")
print("Total number of politeness strategies used in popular side utterances: " + str(num_pop_pos))
print("Total number of impoliteness strategies used in popular side utterances: " + str(num_pop_neg))
print("Total number of politeness strategies used in unpopular side utterances: " + str(num_unpop_pos))
print("Total number of impoliteness strategies used in unpopular side utterances: " + str(num_unpop_neg))

Average number of polite strategies used per utterance by winner: 2.73
Average number of impolite strategies used per utterance by winner: 1.32
Average number of polite strategies used per utterance by loser: 2.74
Average number of impolite strategies used per utterance by loser: 1.31



Total number of winner utterances: 6848
Total number of loser utterances: 6871



Total number of politeness strategies used in winner utterances: 18723
Total number of impoliteness strategies used in winner utterances: 9015
Total number of politeness strategies used in loser utterances: 18851
Total number of impoliteness strategies used in loser utterances: 8982
-------------------------------------------------------------------------
Average number of polite strategies used per utterance by popular side: 2.74
Average number of impolite strategies used per utterance by popular side: 1.30
Average number of polite strategies used per utterance by unpopular side: 2.76
Average number of impolite strategie